<h3 align="center">EL7006 Redes Neuronales y Teoría de la Información para el aprendizaje</h3>
<h3 align="center">Tarea 3: Análisis no lineal de series de tiempo</h3>

In [ ]:
import numpy as np, scipy, scipy.stats
from pandas import Series, DataFrame
import scipy.optimize
%matplotlib inline
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
import subprocess
import platform

"""
Very unsafe way to call tisean methods! Use carefully
"""
def tisean_mutual_information(data_in, max_tau=20):
    np.savetxt("tmp/data_in", data_in)
    if platform.system() == 'Windows':
        pargs = ['./bin/mutual.exe', '-V', '0','-D',str(max_tau),'-o', 'tmp/data_out','tmp/data_in'] 
    else:
        pargs = ['./bin/mutual', '-V', '0','-D',str(max_tau),'-o', 'tmp/data_out','tmp/data_in'] 
    p = subprocess.Popen(pargs)
    p.wait()
    data_out = np.loadtxt("tmp/data_out", comments='#')
    # Return tau and MI
    return data_out[:, 0], data_out[:, 1]

def tisean_FNN(data_in, tau=20, max_dim=10):
    np.savetxt("tmp/data_in", data_in)
    if platform.system() == 'Windows':
        pargs = ['./bin/false_nearest.exe', '-V','0','-d',str(tau),'-m','1','-M','1,'+str(max_dim),'-f','3','-o','tmp/data_out','tmp/data_in'] 
    else:
        pargs = ['./bin/false_nearest', '-V','0','-d',str(tau),'-m','1','-M','1,'+str(max_dim),'-f','3','-o','tmp/data_out','tmp/data_in'] 
    p = subprocess.Popen(pargs)
    p.wait()
    data_out = np.loadtxt("tmp/data_out", comments='#')
    # Return dim and fraction of FNN
    return data_out[:, 0], data_out[:, 1]

def tisean_c2(data_in, tau=20, max_dim=10):
    np.savetxt("tmp/data_in", data_in)
    c2 = list()
    d2 = list()
    if platform.system() == 'Windows': 
        pargs = ['./bin/d2.exe', '-V','0','-d',str(tau),'-#','200','-r','0.1','-M','1,'+str(max_dim),'-o','tmp/data_out','tmp/data_in'] 
    else:
        pargs = ['./bin/d2', '-V','0','-d',str(tau),'-#','200','-r','0.1','-M','1,'+str(max_dim),'-o','tmp/data_out','tmp/data_in'] 
    p = subprocess.Popen(pargs)
    p.wait()
    data_out = np.loadtxt("tmp/data_out.c2", comments='#')
    init_val = data_out[0, 0]
    last_init = 0
    # Search other init vals
    for i in range(1, data_out.shape[0]):
        if data_out[i, 0] == init_val:
            c2.append(data_out[last_init:i, 0:2])
            last_init = i
    c2.append(data_out[last_init:, 0:2])
    data_out = np.loadtxt("tmp/data_out.d2", comments='#')
    init_val = data_out[0, 0]
    last_init = 0
    # Search other init vals
    for i in range(1, data_out.shape[0]):
        if data_out[i, 0] == init_val:
            d2.append(data_out[last_init:i, 0:2])
            last_init = i
    d2.append(data_out[last_init:, 0:2])
    # Return correlation sum and correlation dimension
    return c2, d2

### Rossler attractor

Generated according to:

$$
    \frac{dx}{dt} = -y -z,
$$
$$
    \frac{dy}{dt} = x + ay,
$$
$$
    \frac{dz}{dt} = b + z(x-c),
$$
where $a$, $b$ and $c$ are the parameters of the system

In [ ]:
"""
From http://connor-johnson.com/2014/03/04/fractal-dimension-and-box-counting/
"""
def generate(data_length, odes, state, parameters):
    data = np.zeros([state.shape[0], data_length])
    for i in xrange(5000):
        state = rk4(odes, state, parameters)
    for i in xrange(data_length):
        state = rk4(odes, state, parameters)
        data[:, i] = state
    return data

def rk4(odes, state, parameters, dt=0.01):
    k1 = dt * odes(state, parameters)
    k2 = dt * odes(state + 0.5 * k1, parameters)
    k3 = dt * odes(state + 0.5 * k2, parameters)
    k4 = dt * odes(state + k3, parameters)
    return state + (k1 + 2 * k2 + 2 * k3 + k4) / 6

def rossler_odes((x, y, z), (a, b, c)):
    return np.array([-y - z, x + a * y, b + z * (x - c)])

def rossler_generate(data_length, a=0.2, b=0.2, c=18.0):
    return generate(data_length, rossler_odes, np.array([10.0, 0.0, 0.0]), np.array([a, b, c]))

In [ ]:
attractor = rossler_generate(10000, c=5.7) # c=2.5, 5.7
data = attractor[0]

plt.close()
fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(1,2,1, projection='3d')
ax.plot(attractor[0], attractor[1], attractor[2])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('Rossler attractor')
ax.view_init(elev=30., azim=-45)
ax = fig.add_subplot(1,2,2)
ax.plot(data)
plt.title('x(t)')
ax.set_xlabel('Time')
plt.grid('on')
plt.tight_layout()

### Mutual Information

In [ ]:
tau, mi = tisean_mutual_information(data, max_tau=1000) 
# Find first local minimum
M = 20
for tau_min in xrange(M, len(tau)-M):
    if mi[tau_min] <= np.amin(mi[tau_min-M:tau_min+M]):
        tau_min -=1
        break

print("First local minimum of MI found at lag %d" % tau_min)
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1,1,1)
ax.plot(tau, mi)
ax.set_xlabel('Delay')
ax.set_ylabel('Mutual Information')
ax.plot([tau_min, tau_min], [0.0, np.amax(mi)],'r--')
plt.grid('on')
plt.tight_layout()

### False Nearest Neighbors

In [ ]:
dim, fnn = tisean_FNN(data, tau=1, max_dim=5)
min_fraction = 0.001
for i in range(0, len(dim)):
    if fnn[i] < min_fraction:
        break
fig = plt.figure(figsize=(6, 4))
ax = fig.add_subplot(1,1,1)
ax.plot(dim, fnn)
ax.plot([dim[i], dim[i]], [0, 1], 'r--')
ax.set_xlabel('Dimension')
ax.set_ylabel('Fraction of FNN')
plt.grid('on')
plt.tight_layout()

### Correlation sum and Correlation dimension

$$
    C(m, \varepsilon) = \frac{1}{N_{pairs}} \sum_{j=m}^N \sum_{k<j-w} \Sigma(\varepsilon - |s_i - s_j| )
$$

$$
   C(m, \varepsilon) \approx \varepsilon^{D_2} 
$$

In [ ]:
def moving_average(interval, window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

c2, d2 = tisean_c2(data, tau=1, max_dim=5)

plt.close()
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(1, 2, 1)
for i in range(0, len(c2)):
    ax.plot(c2[i][:, 0], c2[i][:, 1], label='M:'+str(i+1))
ax.set_xlabel('Epsilon')
ax.set_ylabel('Correlation sum')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_aspect(1)
plt.grid('on')
plt.legend(loc=4)
ax = fig.add_subplot(1, 2, 2)
for i in range(0, len(d2)):
    ax.plot(d2[i][:, 0], moving_average(d2[i][:, 1], 20), label='M:'+str(i+1))
ax.set_xlabel('Epsilon')
ax.set_ylabel('Smoothed Correlation dimension')
ax.set_xscale('log')
plt.grid('on')
plt.legend()
plt.tight_layout()

### State space reconstruction

In [ ]:
M = 1
T = 1
ssr = np.zeros(shape=(len(data)-T, 3))
for i in range(0, M):
    ssr[:, i] = np.roll(data, -i*T)[:-T].flatten()

# plot time delay embedding
plt.close()
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(1,2,1, projection='3d')
ax.set_xlabel('X(t)')
ax.set_ylabel('Y(t)')
ax.set_zlabel('Z(t)')
ax.view_init(elev=30., azim=-45)
ax.plot(attractor[0], attractor[1], attractor[2])
plt.title('Original')

ax = fig.add_subplot(1,2,2, projection='3d')
ax.set_xlabel('X(t)')
ax.set_ylabel('Y(t)')
ax.set_zlabel('Z(t)')
ax.view_init(elev=30., azim=-45)
ax.plot(ssr[:, 0], ssr[:, 1], ssr[:, 2])
plt.title('Reconstruction')
plt.tight_layout()